In [3]:
import boto3

client = boto3.client('lambda')
client

In [4]:
with open('brutus/lambda_env/lambda_env.zip', 'rb') as f:
    code = f.read()

resp = client.create_function(
    FunctionName='brutus-func',
    Runtime='python3.6',
    Role='arn:aws:iam::755632011865:role/brutus-lambda-role',
    Handler='lambda_handler.handler',
    Code={'ZipFile': code},
    Description='Lambda Function created by Brutus python package',
    Timeout=10,
    MemorySize=512,
    Publish=True
)

resp

{'CodeSha256': 'HGnBB2JnoRJcrDXDMWoaC3ZO/W5lyLErjY0+OqpGWM0=',
 'CodeSize': 18993815,
 'Description': 'Lambda Function created by Brutus python package',
 'FunctionArn': 'arn:aws:lambda:eu-west-1:755632011865:function:brutus-func',
 'FunctionName': 'brutus-func',
 'Handler': 'lambda_handler.handler',
 'LastModified': '2017-12-18T14:28:59.898+0000',
 'MemorySize': 512,
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '608',
   'content-type': 'application/json',
   'date': 'Mon, 18 Dec 2017 14:29:01 GMT',
   'x-amzn-requestid': 'c796bd9f-e3ff-11e7-9372-ebede17f9d35'},
  'HTTPStatusCode': 201,
  'RequestId': 'c796bd9f-e3ff-11e7-9372-ebede17f9d35',
  'RetryAttempts': 0},
 'Role': 'arn:aws:iam::755632011865:role/brutus-lambda-role',
 'Runtime': 'python3.6',
 'Timeout': 10,
 'TracingConfig': {'Mode': 'PassThrough'},
 'Version': '10'}